# Data Mining Project 2 - Complete Analysis

This notebook delivers a complete project workflow for the **CMI Problematic Internet Use** tabular dataset.

Covered modules:
1. data understanding and preparation
2. outlier analysis and treatment
3. imbalanced learning strategies
4. advanced classification (multiclass target `sii`)
5. model explainability and error analysis
6. additional advanced tasks: regression and clustering


In [ ]:
# Core libraries
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix,
)
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)
from sklearn.inspection import permutation_importance

warnings.filterwarnings('ignore')
sns.set_theme(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 5)
RANDOM_STATE = 42


## 1. Data Loading

Dataset files used:
- `cmi_internet.csv`
- `data_dictionary.csv`


In [ ]:
# Resolve file paths (project-relative first)
data_path_rel = Path('dm2_25_26_dataset_tabular/DM2_project/cmi_internet.csv')
dict_path_rel = Path('dm2_25_26_dataset_tabular/DM2_project/data_dictionary.csv')

data_path_fallback = Path('C:/Users/steve/Downloads/Data_Mining_project2/dm2_25_26_dataset_tabular/DM2_project/cmi_internet.csv')
dict_path_fallback = Path('C:/Users/steve/Downloads/Data_Mining_project2/dm2_25_26_dataset_tabular/DM2_project/data_dictionary.csv')

if data_path_rel.exists():
    data_path = data_path_rel
elif data_path_fallback.exists():
    data_path = data_path_fallback
else:
    raise FileNotFoundError('Dataset file not found')

if dict_path_rel.exists():
    dict_path = dict_path_rel
elif dict_path_fallback.exists():
    dict_path = dict_path_fallback
else:
    dict_path = None

df = pd.read_csv(data_path)
df_original = df.copy()

print('Data path:', data_path)
print('Shape:', df.shape)
df.head(3)


In [ ]:
# Optional: read data dictionary for feature semantics
if dict_path is not None:
    data_dictionary = pd.read_csv(dict_path)
    print('Data dictionary shape:', data_dictionary.shape)
    display(data_dictionary.head(10))
else:
    print('Data dictionary not found.')


## 2. Data Understanding and Preparation


In [ ]:
# Basic structure checks
display(df.info())

print('Number of duplicated rows:', df.duplicated().sum())
print('Number of columns:', df.shape[1])
print('Number of records:', df.shape[0])

# Identify key columns
target_col = 'sii'
id_col = 'id' if 'id' in df.columns else None

print('Target column:', target_col)
print('ID column:', id_col)


In [ ]:
# Missing values profile
missing_count = df.isna().sum()
missing_pct = (missing_count / len(df) * 100).sort_values(ascending=False)

print('Columns with missing values:', int((missing_count > 0).sum()))
print('Top missing columns:')
display(missing_pct.head(20).to_frame('missing_pct'))


In [ ]:
# Class imbalance profile for the target
class_counts = df[target_col].value_counts().sort_index()
class_pct = (class_counts / class_counts.sum() * 100).round(2)

print('Class counts:')
print(class_counts)
print()
print('Class percentages:')
print(class_pct)

fig, ax = plt.subplots(1, 2, figsize=(14, 5))
sns.barplot(x=class_counts.index.astype(int), y=class_counts.values, ax=ax[0], palette='viridis')
ax[0].set_title('Target Distribution (Counts)')
ax[0].set_xlabel('sii class')
ax[0].set_ylabel('count')

ax[1].pie(class_counts.values, labels=class_counts.index.astype(int), autopct='%1.1f%%', startangle=90)
ax[1].set_title('Target Distribution (%)')

plt.tight_layout()
plt.show()


In [ ]:
# Leakage check: PCIAT total score is used to derive severity categories and is strongly tied to sii
pciat_cols = [c for c in df.columns if c.startswith('PCIAT-')]
print('PCIAT columns found:', len(pciat_cols))

if 'PCIAT-PCIAT_Total' in df.columns:
    relation = df[['sii', 'PCIAT-PCIAT_Total']].dropna()
    print('Rows with non-null PCIAT total:', len(relation))
    display(relation.groupby('sii')['PCIAT-PCIAT_Total'].describe()[['count', 'mean', 'min', 'max']])


### 2.1 Feature Sets

We evaluate two feature spaces:
- **full features**: includes `PCIAT-*` variables (high leakage risk)
- **no-leak features**: excludes `PCIAT-*` variables (more realistic prediction setting)


In [ ]:
# Build feature matrices
full_drop = [target_col] + ([id_col] if id_col else [])
X_full = df.drop(columns=full_drop, errors='ignore')

no_leak_drop = full_drop + pciat_cols
X_no_leak = df.drop(columns=no_leak_drop, errors='ignore')

y = df[target_col].astype(int)

print('X_full shape:', X_full.shape)
print('X_no_leak shape:', X_no_leak.shape)


In [ ]:
# Reusable helpers

def make_preprocessor(X):
    numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
    categorical_features = X.select_dtypes(exclude=[np.number]).columns.tolist()

    numeric_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
    ])

    categorical_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ])

    preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
    ])

    return preprocessor


def fit_eval_classifier(model_name, estimator, X_train, y_train, X_test, y_test):
    preprocessor = make_preprocessor(X_train)

    pipe = Pipeline([
        ('preprocess', preprocessor),
        ('model', estimator),
    ])

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    metrics = {
        'model': model_name,
        'accuracy': accuracy_score(y_test, y_pred),
        'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
        'macro_f1': f1_score(y_test, y_pred, average='macro'),
        'weighted_f1': f1_score(y_test, y_pred, average='weighted'),
    }

    return pipe, y_pred, metrics


def crossval_macro_f1(estimator, X, y, folds=3):
    preprocessor = make_preprocessor(X)
    pipe = Pipeline([
        ('preprocess', preprocessor),
        ('model', estimator),
    ])

    cv = StratifiedKFold(n_splits=folds, shuffle=True, random_state=RANDOM_STATE)
    scoring = {
        'macro_f1': 'f1_macro',
        'bal_acc': 'balanced_accuracy',
        'acc': 'accuracy',
    }

    scores = cross_validate(pipe, X, y, cv=cv, scoring=scoring)
    return {
        'cv_macro_f1_mean': scores['test_macro_f1'].mean(),
        'cv_macro_f1_std': scores['test_macro_f1'].std(),
        'cv_bal_acc_mean': scores['test_bal_acc'].mean(),
        'cv_acc_mean': scores['test_acc'].mean(),
    }


## 3. Baseline Classification


In [ ]:
# Split data for both tracks
Xf_train, Xf_test, yf_train, yf_test = train_test_split(
    X_full, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

Xn_train, Xn_test, yn_train, yn_test = train_test_split(
    X_no_leak, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print('Full train/test:', Xf_train.shape, Xf_test.shape)
print('No-leak train/test:', Xn_train.shape, Xn_test.shape)


In [ ]:
# Baseline models
base_models = {
    'dummy_prior': DummyClassifier(strategy='prior', random_state=RANDOM_STATE),
    'logistic_regression': LogisticRegression(max_iter=1200),
    'decision_tree': DecisionTreeClassifier(random_state=RANDOM_STATE, min_samples_leaf=10),
    'random_forest': RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1),
}

rows = []
fitted_models = {}
predictions = {}

# Evaluate on FULL feature space
for name, model in base_models.items():
    fit_pipe, y_pred, m = fit_eval_classifier(
        model_name=f'full::{name}',
        estimator=model,
        X_train=Xf_train,
        y_train=yf_train,
        X_test=Xf_test,
        y_test=yf_test,
    )
    rows.append(m)
    fitted_models[m['model']] = fit_pipe
    predictions[m['model']] = y_pred

# Evaluate on NO-LEAK feature space
for name, model in base_models.items():
    fit_pipe, y_pred, m = fit_eval_classifier(
        model_name=f'noleak::{name}',
        estimator=model,
        X_train=Xn_train,
        y_train=yn_train,
        X_test=Xn_test,
        y_test=yn_test,
    )
    rows.append(m)
    fitted_models[m['model']] = fit_pipe
    predictions[m['model']] = y_pred

baseline_comparison = pd.DataFrame(rows).sort_values('macro_f1', ascending=False)
display(baseline_comparison)


In [ ]:
# Cross-validation on the no-leak setting for more robust model selection
cv_rows = []

for name, model in base_models.items():
    cv_stats = crossval_macro_f1(model, X_no_leak, y, folds=3)
    cv_rows.append({'model': f'noleak::{name}', **cv_stats})

cv_df = pd.DataFrame(cv_rows).sort_values('cv_macro_f1_mean', ascending=False)
display(cv_df)


## 4. Outlier Analysis and Treatment

Outlier detection is done on numeric training features using `IsolationForest`, then we compare model quality before/after filtering outliers.


In [ ]:
# Detect outliers on training numeric data (no-leak track)
num_cols_noleak = Xn_train.select_dtypes(include=[np.number]).columns.tolist()
train_num = Xn_train[num_cols_noleak].copy()
train_num = train_num.fillna(train_num.median())

iso = IsolationForest(contamination=0.03, random_state=RANDOM_STATE)
iso_flags = iso.fit_predict(train_num)
inlier_mask = (iso_flags == 1)

Xn_train_inlier = Xn_train.loc[inlier_mask]
yn_train_inlier = yn_train.loc[inlier_mask]

print('Original no-leak training size:', len(Xn_train))
print('Inlier training size:', len(Xn_train_inlier))
print('Removed outliers:', int((~inlier_mask).sum()))


In [ ]:
# Compare outlier handling impact with Decision Tree and Random Forest
outlier_eval_rows = []

for model_name, estimator in {
    'decision_tree': DecisionTreeClassifier(random_state=RANDOM_STATE, min_samples_leaf=10),
    'random_forest': RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1),
}.items():
    _, _, m_clean = fit_eval_classifier(
        f'noleak_clean::{model_name}', estimator, Xn_train, yn_train, Xn_test, yn_test
    )
    _, _, m_inlier = fit_eval_classifier(
        f'noleak_inlier::{model_name}', estimator, Xn_train_inlier, yn_train_inlier, Xn_test, yn_test
    )
    outlier_eval_rows.extend([m_clean, m_inlier])

outlier_df = pd.DataFrame(outlier_eval_rows).sort_values('macro_f1', ascending=False)
display(outlier_df)


## 5. Imbalanced Learning Strategies (No-Leak Track)


In [ ]:
# Algorithm-level balancing with class weights
weighted_models = {
    'noleak::logreg_weighted': LogisticRegression(max_iter=1200, class_weight='balanced'),
    'noleak::dtree_weighted': DecisionTreeClassifier(random_state=RANDOM_STATE, min_samples_leaf=10, class_weight='balanced'),
    'noleak::rf_weighted': RandomForestClassifier(
        n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1, class_weight='balanced_subsample'
    ),
}

weighted_rows = []
weighted_fitted = {}
weighted_preds = {}

for model_name, estimator in weighted_models.items():
    fit_pipe, y_pred, m = fit_eval_classifier(
        model_name=model_name,
        estimator=estimator,
        X_train=Xn_train,
        y_train=yn_train,
        X_test=Xn_test,
        y_test=yn_test,
    )
    weighted_rows.append(m)
    weighted_fitted[model_name] = fit_pipe
    weighted_preds[model_name] = y_pred

weighted_df = pd.DataFrame(weighted_rows).sort_values('macro_f1', ascending=False)
display(weighted_df)


In [ ]:
# Manual random resampling (train only) so notebook runs without extra libraries

def random_oversample(X_in, y_in, random_state=42):
    rng = np.random.default_rng(random_state)
    work = X_in.copy()
    work['_target_'] = y_in.values

    class_counts = work['_target_'].value_counts()
    majority_size = class_counts.max()

    parts = []
    for cls, count in class_counts.items():
        cls_rows = work[work['_target_'] == cls]
        if count < majority_size:
            extra_idx = rng.choice(cls_rows.index.to_numpy(), size=majority_size - count, replace=True)
            cls_rows = pd.concat([cls_rows, cls_rows.loc[extra_idx]], axis=0)
        parts.append(cls_rows)

    sampled = pd.concat(parts, axis=0).sample(frac=1.0, random_state=random_state)
    return sampled.drop(columns=['_target_']), sampled['_target_'].astype(int)


def random_undersample(X_in, y_in, random_state=42):
    work = X_in.copy()
    work['_target_'] = y_in.values

    class_counts = work['_target_'].value_counts()
    minority_size = class_counts.min()

    parts = []
    for cls in class_counts.index:
        cls_rows = work[work['_target_'] == cls].sample(n=minority_size, replace=False, random_state=random_state)
        parts.append(cls_rows)

    sampled = pd.concat(parts, axis=0).sample(frac=1.0, random_state=random_state)
    return sampled.drop(columns=['_target_']), sampled['_target_'].astype(int)


X_over, y_over = random_oversample(Xn_train, yn_train, random_state=RANDOM_STATE)
X_under, y_under = random_undersample(Xn_train, yn_train, random_state=RANDOM_STATE)

print('Original class distribution:')
print(yn_train.value_counts().sort_index())
print()
print('Oversampled class distribution:')
print(y_over.value_counts().sort_index())
print()
print('Undersampled class distribution:')
print(y_under.value_counts().sort_index())


In [ ]:
# Evaluate resampling strategies with Random Forest
resampling_rows = []

rf_est = RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1)

_, y_pred_over, m_over = fit_eval_classifier(
    model_name='noleak::rf_oversampled',
    estimator=rf_est,
    X_train=X_over,
    y_train=y_over,
    X_test=Xn_test,
    y_test=yn_test,
)

_, y_pred_under, m_under = fit_eval_classifier(
    model_name='noleak::rf_undersampled',
    estimator=rf_est,
    X_train=X_under,
    y_train=y_under,
    X_test=Xn_test,
    y_test=yn_test,
)

resampling_rows.extend([m_over, m_under])
resampling_df = pd.DataFrame(resampling_rows).sort_values('macro_f1', ascending=False)
display(resampling_df)


## 6. Final Classification Comparison and Diagnostics


In [ ]:
# Collect no-leak models in one comparison table
noleak_baselines = baseline_comparison[baseline_comparison['model'].str.startswith('noleak::')]

all_cls_results = pd.concat(
    [noleak_baselines, weighted_df, resampling_df],
    ignore_index=True
).sort_values('macro_f1', ascending=False)

display(all_cls_results)

plot_data = all_cls_results[['model', 'macro_f1', 'balanced_accuracy']].set_index('model')
plot_data.plot(kind='bar', figsize=(14, 6), colormap='Set2')
plt.title('No-Leak Classification Comparison')
plt.ylabel('score')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
# Refit the best no-leak model and inspect class-level performance
best_model_name = all_cls_results.iloc[0]['model']

model_factory = {
    'noleak::dummy_prior': DummyClassifier(strategy='prior', random_state=RANDOM_STATE),
    'noleak::logistic_regression': LogisticRegression(max_iter=1200),
    'noleak::decision_tree': DecisionTreeClassifier(random_state=RANDOM_STATE, min_samples_leaf=10),
    'noleak::random_forest': RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1),
    'noleak::logreg_weighted': LogisticRegression(max_iter=1200, class_weight='balanced'),
    'noleak::dtree_weighted': DecisionTreeClassifier(random_state=RANDOM_STATE, min_samples_leaf=10, class_weight='balanced'),
    'noleak::rf_weighted': RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1, class_weight='balanced_subsample'),
    'noleak::rf_oversampled': RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1),
    'noleak::rf_undersampled': RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1),
}

# Use matching train data for resampling variants
if best_model_name == 'noleak::rf_oversampled':
    X_train_best, y_train_best = X_over, y_over
elif best_model_name == 'noleak::rf_undersampled':
    X_train_best, y_train_best = X_under, y_under
else:
    X_train_best, y_train_best = Xn_train, yn_train

best_estimator = model_factory[best_model_name]
best_pipe, best_pred, best_metrics = fit_eval_classifier(
    model_name=best_model_name,
    estimator=best_estimator,
    X_train=X_train_best,
    y_train=y_train_best,
    X_test=Xn_test,
    y_test=yn_test,
)

print('Best model:', best_model_name)
print(best_metrics)
print()
print(classification_report(yn_test, best_pred, digits=3))

labels = sorted(y.unique())
cm = confusion_matrix(yn_test, best_pred, labels=labels)
plt.figure(figsize=(7, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title(f'Confusion Matrix - {best_model_name}')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.show()


In [ ]:
# Explainability: permutation feature importance on best no-leak model
perm = permutation_importance(
    estimator=best_pipe,
    X=Xn_test,
    y=yn_test,
    scoring='f1_macro',
    n_repeats=5,
    random_state=RANDOM_STATE,
    n_jobs=-1,
)

fi = pd.DataFrame({
    'feature': Xn_test.columns,
    'importance_mean': perm.importances_mean,
    'importance_std': perm.importances_std,
}).sort_values('importance_mean', ascending=False)

display(fi.head(20))

plt.figure(figsize=(12, 6))
sns.barplot(data=fi.head(20), x='importance_mean', y='feature', palette='crest')
plt.title('Top 20 Permutation Importances (Macro-F1)')
plt.xlabel('Importance (mean decrease)')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()


## 7. Advanced Regression Task (Extra)

We model a continuous target: `SDS-SDS_Total_T`.

Leakage prevention for regression:
- remove `SDS-SDS_Total_Raw` (directly related score)
- remove `PCIAT-*` columns


In [ ]:
reg_target = 'SDS-SDS_Total_T'

reg_df = df[df[reg_target].notna()].copy()
reg_drop = [id_col, reg_target, 'SDS-SDS_Total_Raw'] + pciat_cols
reg_drop = [c for c in reg_drop if c is not None]

X_reg = reg_df.drop(columns=reg_drop, errors='ignore')
y_reg = reg_df[reg_target].astype(float)

Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=RANDOM_STATE
)

print('Regression rows used:', len(reg_df))
print('Regression train/test:', Xr_train.shape, Xr_test.shape)


In [ ]:
# Regression helper

def fit_eval_regressor(model_name, estimator, X_train, y_train, X_test, y_test):
    preprocessor = make_preprocessor(X_train)

    pipe = Pipeline([
        ('preprocess', preprocessor),
        ('model', estimator),
    ])

    pipe.fit(X_train, y_train)
    pred = pipe.predict(X_test)

    rmse = mean_squared_error(y_test, pred) ** 0.5

    metrics = {
        'model': model_name,
        'MAE': mean_absolute_error(y_test, pred),
        'RMSE': rmse,
        'R2': r2_score(y_test, pred),
    }

    return pipe, pred, metrics


reg_models = {
    'dummy_mean': DummyRegressor(strategy='mean'),
    'rf_regressor': RandomForestRegressor(n_estimators=250, random_state=RANDOM_STATE, n_jobs=-1),
    'gbr_regressor': GradientBoostingRegressor(random_state=RANDOM_STATE),
}

reg_rows = []
for name, model in reg_models.items():
    _, _, m = fit_eval_regressor(name, model, Xr_train, yr_train, Xr_test, yr_test)
    reg_rows.append(m)

reg_results = pd.DataFrame(reg_rows).sort_values('RMSE')
display(reg_results)


## 8. Clustering Analysis (Extra)

Unsupervised clustering on no-leak **numeric** features:
- median imputation + standardization
- silhouette-based selection of `k`
- 2D PCA visualization


In [ ]:
num_cols = X_no_leak.select_dtypes(include=[np.number]).columns.tolist()
X_num = X_no_leak[num_cols].copy()

imp = SimpleImputer(strategy='median')
X_num_imp = imp.fit_transform(X_num)

sc = StandardScaler()
X_num_scaled = sc.fit_transform(X_num_imp)

sil_rows = []
for k in range(2, 7):
    km = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=20)
    labels = km.fit_predict(X_num_scaled)
    sil = silhouette_score(X_num_scaled, labels)
    sil_rows.append({'k': k, 'silhouette': sil})

sil_df = pd.DataFrame(sil_rows).sort_values('silhouette', ascending=False)
display(sil_df)

best_k = int(sil_df.iloc[0]['k'])
print('Best k from silhouette:', best_k)


In [ ]:
# Fit clustering with best k and visualize clusters in 2D
km_best = KMeans(n_clusters=best_k, random_state=RANDOM_STATE, n_init=20)
cluster_labels = km_best.fit_predict(X_num_scaled)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_num_scaled)

plot_df = pd.DataFrame({
    'PC1': X_pca[:, 0],
    'PC2': X_pca[:, 1],
    'cluster': cluster_labels,
    'sii': y.values,
})

plt.figure(figsize=(10, 6))
sns.scatterplot(data=plot_df, x='PC1', y='PC2', hue='cluster', palette='tab10', s=35, alpha=0.8)
plt.title('KMeans Clusters (PCA projection)')
plt.tight_layout()
plt.show()

cluster_sii = pd.crosstab(plot_df['cluster'], plot_df['sii'], normalize='index').round(3)
print('Class composition per cluster (row-normalized):')
display(cluster_sii)


## 9. Final Conclusions

Key findings:
- `sii` is strongly imbalanced, so `macro_f1` and `balanced_accuracy` are required.
- `PCIAT-*` variables create strong leakage risk when predicting `sii`; no-leak analysis is more realistic.
- in the no-leak setting, model performance is significantly harder and closer to baseline, which is expected.
- outlier filtering can be evaluated but may not always improve multiclass metrics.
- class weighting and sampling strategies should be compared empirically because gains vary by model and class.
- additional regression and clustering analyses provide broader project coverage and exploratory insight.

Possible next upgrades:
1. hyperparameter tuning with `GridSearchCV`/`RandomizedSearchCV`
2. advanced resampling (SMOTE/ADASYN) if `imbalanced-learn` is installed
3. threshold/cost-sensitive optimization targeting minority classes
